In [1]:
import sklearn
import nltk

from generateWordFrequency import *
from naiveBayes import *
from fileWriteFunctions import *

In [2]:
AllClasses = ['story', 'ask_hn', 'show_hn', 'poll']
delta = 0.5
appendClassPrefix = 'prob_'

In [3]:
trainData = getDataframe(2018)

In [4]:
trainData = addTokenizedColumnofTitle(trainData,None,False)

In [5]:
trainWords = getWordFrequencyDataframe(trainData,AllClasses)

In [6]:
trainWords.head()

,top,hacker,news,book,in,2017,beijing,enjoy,best,winter,...,400gb,shareseer,öyster,face-detecting,rainer,rilke,computing.foundation,integer32,nichols,goulding
story,1277,1048,1424,1283,33965,2091,81,49,1716,147,...,1,1,1,1,1,1,1,1,1,2
ask_hn,37,86,111,249,1420,22,1,10,648,2,...,0,0,0,0,0,0,0,0,0,0
show_hn,68,100,184,77,1149,6,0,1,57,2,...,0,0,0,0,0,0,0,0,0,0
poll,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
filteredWordList = getWordListBasedOnPercent(trainWords,Percent = 25)
print(filteredWordList[0],len(filteredWordList))

the 23156


In [8]:
trainData = addTokenizedColumnofTitle(trainData,filteredWordList,False)
trainWords = getWordFrequencyDataframe(trainData,AllClasses)
trainWords.head()

,registrars,simplecoin,2min,37000,altco,plirus,bsp,self-assembly,wav2letter,toshi,...,400gb,shareseer,öyster,face-detecting,rainer,rilke,computing.foundation,integer32,nichols,goulding
story,0,1,2,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,2
ask_hn,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
show_hn,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
poll,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
trainData.head()

,Title,Post Type,Number of Comments,Points,Author,tokenized_title
0,Top Hacker News Books in 2017,story,0.0,1,0x54MUR41,[]
1,Beijing enjoys best winter air quality in five...,story,0.0,2,gpetukhov,[]
2,Ask HN: Any domain name registrars that don't ...,ask_hn,2.0,1,glockenspielen,[registrars]
3,Controversial Therapy Has Led to Death Threats...,story,0.0,2,cpncrunch,[]
4,Ruby 3x3 – Ruby 3 Will Be 3 Times Faster – Wha...,story,0.0,2,geraldbauer,[]


In [10]:
priorProbabilities = getPriorProbabilities(trainData)
trainWords = obtainDataframeWithClassProbabilities(trainWords, AllClasses, delta, appendClassPrefix)
writeModel(trainWords,'wordlength-model.txt',AllClasses,appendClassPrefix)
trainWords = renameModelRows(trainWords, AllClasses, appendClassPrefix)
model = [trainWords, priorProbabilities]

In [12]:
testData = getDataframe(2019)
testData = addTokenizedColumnofTitle(testData,filteredWordList,False)
testData = testData.reset_index()
testData = generateCondClassProb(testData, model)
testData['predicted'] = generatePrediction(testData,AllClasses)
testResults = comparePredictions(testData,AllClasses)
writeDataframe(testResults,'wordlength-result.txt')
check = testResults.comparision.value_counts()
accuracy = check[True]/(check[True]+check[False])
print(accuracy)
df_confusion = pd.crosstab(testResults['Post Type'], testResults['predicted'])
df_confusion['poll'] = 0
print(df_confusion)
df_confusion['precision'] = 0.0
df_confusion['recall'] = 0.0
#df['F1_score'] = 0.0
transpose_matrix = df_confusion.transpose()
for i in transpose_matrix.columns:
    if np.sum(df_confusion[i])==0:
        total = 1
    else:
        total = np.sum(df_confusion[i])
    df_confusion['precision'][i] = df_confusion[i][i]/total
    df_confusion['recall'][i] = df_confusion[i][i]/np.sum(transpose_matrix[i])
df_confusion['F1_score'] = 2/(1/df_confusion['precision']+1/df_confusion['recall'])
print(df_confusion)

0.9244980505046825
predicted  ask_hn  show_hn   story  poll
Post Type                               
ask_hn          0        3    5451     0
poll            0        0       6     0
show_hn         0       61    4842     0
story           4       54  126794     0
predicted  ask_hn  show_hn   story  poll  precision    recall  F1_score
Post Type                                                              
ask_hn          0        3    5451     0   0.000000  0.000000  0.000000
poll            0        0       6     0   0.000000  0.000000  0.000000
show_hn         0       61    4842     0   0.516949  0.012441  0.024298
story           4       54  126794     0   0.924876  0.999543  0.960761


/home/paras/anaconda3/envs/cu/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/paras/anaconda3/envs/cu/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
